In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import h5py
import math
import pickle
from datetime import datetime
from numba import cuda 

%matplotlib inline

In [2]:
import sys  
sys.path.insert(0, '..')

In [3]:
from tiling.read_tiles import TissueDataset, load_color_normalization_values

In [4]:
# Code snippet nvidia gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
HDF5_FOLDER = '/media/nico/data/fourthbrain/project/training_CAMELYON16'
green_layer_only = False
color_normalization_file="/home/nico/Documents/FourthBrainBreastCancer/tiling/CAMELYON16_color_normalization.json"

In [7]:
train_data = TissueDataset(HDF5_FOLDER,  percentage=0.8, first_part=True, crop_size=256)
val_data = TissueDataset(HDF5_FOLDER, percentage=0.2, first_part=False, crop_size=256)

In [ ]:
batch_size_neg=50
batch_size_pos=50
batches_per_train_epoch = 100
batches_per_val_epoch = 50
MAX_EPOCHS = 10

In [ ]:
def history_to_pickle(history, filepath):
    with open(filepath, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)
        
def pickle_to_history(filepath):
    history = pickle.load(open(filepath, "rb"))
    return history

In [ ]:
def plot_history(history):
    # list all data in history
    print(history.keys())
    # summarize history for accuracy
    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
# FEATURE EXTRACTOR MODEL

In [ ]:
model = keras.applications.InceptionV3(input_shape=(256,256,3),
                                        include_top=True,
                                        classes = 1,
                                        classifier_activation="sigmoid",
                                        weights=None)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [ ]:
model_type = 'train_no_filters_rgb_normalized_level_3'
model_hdf5 = '/media/nico/data/fourthbrain/project/model_testing/model_{}_'.format(model_type) + datetime.now().strftime("%Y%m%d-%H%M%S") + '.hdf5'
filepath = '/media/nico/data/fourthbrain/project/model_testing/{}.pkl'.format(model_type)
log_dir = "/media/nico/data/fourthbrain/project/model_testing/" + datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_hdf5, monitor='loss',verbose=1, save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, 
                           verbose=1, mode='max', min_lr=1e-5)

#all callbacks
callbacks = [tensorboard_callback, model_checkpoint, reduce_lr]  

In [ ]:
now1 = datetime.now()
    
history = model.fit(x=train_data.generator(batch_size_neg, batch_size_pos, True, green_layer_only=False, color_normalization_file=None),
                    validation_data=val_data.generator(batch_size_neg, batch_size_pos, False, green_layer_only=False, color_normalization_file=None),
                    epochs=MAX_EPOCHS,
                    steps_per_epoch = batches_per_train_epoch,
                    validation_steps = batches_per_val_epoch,
                    callbacks=callbacks
                   )

dtme = now1.strftime('%m%d%Y_%H%M%S.pkl')
now2 = datetime.now()

In [ ]:
print('duration: {} - path: {}'.format(now2 - now1, filepath))
history_to_pickle(history, filepath)

In [ ]:
model.save('model_inceptionv3.hdf5')